In [1]:

%pprint
import sys
import os.path as osp

executable_path = sys.executable; scripts_folder = osp.join(osp.dirname(executable_path), 'Scripts')
py_folder = osp.abspath('../py'); ffmpeg_folder = r'C:\ffmpeg\bin'
if (scripts_folder not in sys.path): sys.path.insert(1, scripts_folder)
if (py_folder not in sys.path): sys.path.insert(1, py_folder)
if (ffmpeg_folder not in sys.path): sys.path.insert(1, ffmpeg_folder)
from jobpostlib import (cu, datetime, duration, freq, hau, humanize, nu, scrfcu, slrcu, su, t0, time, wsu, speech_engine)
import os

Pretty printing has been turned OFF
======== Neo4j/4.4.7 ========
Utility libraries created in 10 seconds



----

In [7]:

wsu.save_email_to_file('indeed', verbose=True)

['powershell.exe', '-ExecutionPolicy', 'Bypass', '-Command', 'Start-Process "C:\\Program Files\\Notepad++\\notepad++.exe" -ArgumentList "C:\\Users\\daveb\\OneDrive\\Documents\\GitHub\\job-hunting\\data\\html\\indeed_email.html"']


In [8]:

driver = wsu.get_driver(verbose=False); speech_engine.say('Browser available'); speech_engine.runAndWait()
viewjob_links_set = set(); location_strs_set = set()
file_path = '../data/html/indeed_email.html'
page_soup = wsu.get_page_soup(file_path)
rows_css = 'table > tbody > tr'
cells_css = 'td > a > table > tbody > tr:nth-child(3) > td'
row_soups_list = page_soup.select(rows_css)
for row_soup in row_soups_list:
    cell_soups_list = row_soup.select(cells_css)
    if cell_soups_list:
        location_str = cell_soups_list[0].text.strip()
        if any(map(lambda x: x in location_str, ['Massachusetts', 'United States', 'Clinton, MA'])):
            location_strs_set.add(location_str)
            page_soup = list(cell_soups_list[0].parent.previous_siblings)[1]
            
            # Get viewjob links
            for link_soup in page_soup.select('a[href^="https://www.indeed.com/rc/clk/dl?jk="]'):
                new_url = su.clean_indeed_url(link_soup['href'], driver, verbose=False)
                viewjob_links_set.add(new_url)
            
            # Get pagead links and convert them to viewjob links
            for link_soup in page_soup.select('a[href^="https://www.indeed.com/pagead/clk"]'):
                new_url = su.clean_indeed_url(link_soup['href'], driver, verbose=False)
                viewjob_links_set.add(new_url)

speech_str = f'I found {len(viewjob_links_set)} urls (in {nu.conjunctify_nouns(location_strs_set)}) in indeed_email.html'; print(speech_str)

I found 9 urls (in United States, Clinton, MA • Hybrid remote, Clinton, MA, and United States • Hybrid remote) in indeed_email.html


In [9]:

files_list = []; close_notices = True
for url_str in viewjob_links_set:
    file_node_dict, files_list = su.load_indeed_posting_url(
        viewjob_url=url_str, driver=driver, files_list=files_list, close_notices=close_notices, slowed_for_readability=False, search_type='Indeed 01510', verbose=False
    )
    close_notices = False
speech_str = f'Fileing {len(files_list)} postings out of {len(viewjob_links_set)} urls complete. Delete the email.'; print(speech_str)
speech_engine.say(speech_str); speech_engine.runAndWait()

Fileing 2 postings out of 9 urls complete. Delete the email.


In [10]:

import re

t1 = time.time()
try: driver.close()
except Exception as e: print(f'{e.__class__.__name__} error: {str(e).strip()}')
cu.ensure_navigableparent('END', verbose=False)
for file_name in files_list:
    file_path = os.path.join(cu.SAVES_HTML_FOLDER, file_name)
    
    # Delete each of the previous siblings of the target_div that are <div> elements and prettify the HTML
    wsu.clean_job_posting(file_path)
    
    page_soup = wsu.get_page_soup(file_path)
    row_div_list = page_soup.find_all(name='div', id='jobDescriptionText')
    assert row_div_list, f'{file_name} is missing <div id="jobDescriptionText">'
    for target_div in row_div_list:
        child_strs_list = hau.get_navigable_children(target_div, [])
        assert child_strs_list, f'{file_name} is missing its child strings'
        cu.populate_from_child_strings(child_strs_list, file_name, verbose=False)
duration_str = humanize.precisedelta(time.time() - t1, minimum_unit='seconds', format='%0.0f')
speech_str = f'Populating {len(files_list)} out of {len(viewjob_links_set)} postings completed in {duration_str}'; speech_engine.say(speech_str); speech_engine.runAndWait()
files_list

['7b208bae8a305f32_Picker_Packer_Warehouse_Associate_Clinton_MA_Indeed_com.html', '89626071a9cc0a2c_Customer_Support_Representative_II_United_States_Indeed_com.html']